# Imports

### TensorFlow

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

2025-03-18 12:17:56.575801: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-18 12:17:56.583842: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-18 12:17:56.604984: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742300276.627600   41930 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742300276.635454   41930 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742300276.652804   41930 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Processing Data

In [3]:
dataset = pd.read_csv("../datasets/dataset.csv")

train_texts, test_texts, train_labels, test_labels = train_test_split(
    dataset["Text"], dataset["Label"], test_size=0.2, random_state=42, stratify=dataset["Label"]
)

In [4]:
vectorizer = TfidfVectorizer(max_features=5000)

X_train = vectorizer.fit_transform(train_texts).toarray()
X_test = vectorizer.fit_transform(test_texts).toarray()

y_train = np.array(train_labels)
y_test = np.array(test_labels)

# Neural Network

In [5]:
model = Sequential([
    Dense(128, activation="relu", input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.2),

    Dense(64, activation="relu", kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.2),

    Dense(32, activation="relu", kernel_regularizer=l2(0.002)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(1, activation="sigmoid") 
])

/home/jvale/miniconda3/envs/Cenas/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-03-18 12:18:16.480506: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [ ]:
optimizer = Adam(learning_rate=0.005)

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stop])

Epoch 1/20


2025-03-18 12:18:20.545289: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1237680000 exceeds 10% of free system memory.


1929/1934 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8081 - loss: 0.7354

In [7]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

484/484 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5120 - loss: 1.4304
Loss: 1.4527, Accuracy: 0.5038
